In [30]:
import random, math
import numpy as np

def sigmoid(x):
    return 1/(1+np.exp(-x))

def swish(x):
    return x*sigmoid(x)

def relu(x):
    return np.maximum(0,x)

def activation_function(z,act):
    if act=="sigmoid":
        return sigmoid(z)
    elif act == "swish":
        return swish(z)
    elif act == "relu":
        return relu(z)
    elif act=="tanh":
        return np.tanh(z)
    
class Layer():
    def __init__(self,input_size,output_size,act=None):
        self.size = input_size
        self.output_size = output_size
        self.activation = act
        self.weights = np.random.uniform(-1,1,(self.output_size,self.size))    
        self.bias = np.random.uniform(-1,1,(output_size,1))
        self.A = np.random.uniform(-1,1,(output_size,1))
        self.Z = np.random.uniform(-1,1,(output_size,1))
        
    def forward_propagation(self, input_data):
        self.Z = np.dot(self.weights,input_data)+self.bias
        self.A = activation_function(self.Z, self.activation)
        
    def derMSE(self, target):
        return 2*(self.A - target)
    
    def descent(self, input_data, gradient, learningRate):
        
        if(self.activation == "tanh"):
            derZ = 1 - np.power(self.A, 2)
        elif(self.activation == "swish"):
            derZ = swish(self.Z) + sigmoid(self.Z) * (1 - swish(self.Z))
        reps = (self.weights.shape[0], 1)
        derWeights = np.tile(input_data.transpose(), reps)        # Size -> a(L-1)*a(L)
        """derWeights is a matrix with derivatives of Z WRT weights, which is transposed inputs, 
        repeated in rows n-times, where n is number of neurons.
        
    Example:
    
        input_data = [
            [2],
            [1],
            [0]
        ]
        
        derWeights = [
            [2, 1, 0],
            [2, 1, 0],
            [2, 1, 0],
            ... n-rows
        ]
        
        """

        
        """ The below part adds the gradient to the derivative of A WRT input_data and passes this new
        gradient through return, to be used as the gradient for next layer's descent.
            dA/dX is made with 2 steps: adding the backprop gradient to the derivative of A WRT Z and then adding
        the derivative of Z WRT input_data (chain rule).
        
        1.
        Since A is a matrix shaped Nx1, where N is the number of outputs, the receiving gradient from the upper layer
        must be the same shape. Therefore we can multiply the gradient and the derivative together element-wise.
        
    Example:
        
         derZ = [3x1]
         gradient = [3x1]
         firstGrad = [3x1] *(elementwise) [3x1] = [3x1]
        
        """
        firstGrad = np.multiply(gradient, derZ)             # ∂C/∂a(L) * ∂(act)/∂Z    a(L)*1 * a(L)*1 = a(L)*1
        
        """
        2.
        What's left is adding the gradient of Z WRT input_data. 
        
        This turns out to be the weights matrix. Now we have to multiply the firstGrad gradient to these weights 
        element-wise but since the firstGrad is Nx1 shape and the weights are NxM, where M are the features, 
        we need to reshape the gradient matrix to match the weights matrix by cloning gradient's columns:
        """
        secondGrad = np.tile(firstGrad, (1, self.weights.shape[1]))          #Size -> a(L)*1 -> a(L)*a(L-1)
        
        """
        Finally we multiply (E-W) secondGrad to the weights matrix:
        """
        finalGrad = np.multiply(self.weights,secondGrad)         #Size -> a(L)*a(L-1) * a(L)*a(L-1) = a(L)*a(L-1)
        
        """But because same inputs are multiplied with many weights, we can sum those weights together. It turns out
        that we can sum columns to do that"""
        
        finalGrad = np.sum(finalGrad, axis=0, keepdims=True)       #Size -> 1*a(L-1) 
        derBias = 1
        
        weightGrad = np.multiply(derWeights, np.tile(firstGrad, (1, self.weights.shape[1])))
        self.weights = self.weights - learningRate * weightGrad
        self.bias = self.bias - learningRate * firstGrad
        
        """We return transposed matrix, because we desire inputs with a shape of Nx1 and right now finalGrad is 
        transposed"""
        return finalGrad.transpose()     # Size -> a(L-1)*1
    
        
class NeuralNetwork():
    def __init__(self):
        self.layers=[]
        self.epochs=10
        self.learning_rate = 0.008
    
    def add_layer(self,input_size,output_size,activation=None):
        new_layer = Layer(input_size,output_size,activation)
        self.layers.append(new_layer)
        
    def forward_propagation(self,layer_no):
        current_layer = self.layers[layer_no-1]
        prev_layer = self.layers[layer_no-2]
        act = current_layer.activation
        input_data = prev_layer.A
        self.Z = np.dot(weights,input_data)+self.bias
        result = activation_function(self.Z,act)    # array containing neuron values
        current_layer.A = result            #After forward propogation, fills in the neurons in that layer
        return result
    
    def full_forward_propagation(self, input_data):
        #print("layer 0 forward_propagation")
        
        self.layers[0].forward_propagation(input_data)        # From input data to first layer
        for i in range(1, len(self.layers)):
            #print("layer " + str(i) + " forward_propagation")
            
            self.layers[i].forward_propagation(self.layers[i-1].A)      #From layer i-1 to layer i 
        return self.layers[len(self.layers)-1].A
    
    def back_propagation(self, input_data, target):
        gradient = self.layers[len(self.layers)-1].derMSE(target)     # ∂C/∂a(L)   Size -> a(L)*1
        for i in range(0, len(self.layers)-1):
            index = len(self.layers)-1 - i
            #print("Layer " + str(index) + " backpropagation")
            gradient = self.layers[index].descent(self.layers[index-1].A, gradient, self.learning_rate)   #a(L)*1
        self.layers[0].descent(input_data, gradient, self.learning_rate)
            
            
    def predict(self,test_data):
        self.layers[0].forward_propagation(test_data)
        for i in range(1, len(self.layers)):
            self.layers[i].forward_propagation(self.layers[i-1].A)
        return self.layers[len(self.layers)-1].A

train_data = np.array([[[3],[2],[1]],
                      [[5],[1],[1]],
                      [[5],[0],[0]],
                      [[3],[0],[1]],
                      [[4],[-1],[2]],
                      [[3],[-1],[5]],
                      [[3],[-1],[11]],
                      [[3],[1],[3]],
                      [[-1],[1],[1]],
                      [[2],[0],[1]],
                      [[0],[0],[1]],
                      [[3],[0],[5]],
                      [[2],[-1],[6]],
                      [[1],[1],[-4]],
                      [[0],[0],[-1]],
                      [[2],[1],[-4]],
                      [[2],[1],[-1]],
                      [[1],[0],[-1]],
                      [[10],[4],[-2]]])

"""Here we generate target_data set using formula 2*x0 + -4*x1 + -x2"""
target_data = []
for i in range(0, len(train_data)):
    if train_data[i][0]*2 + train_data[i][1]*-4 + train_data[i][2]*1 > 0:
        target_data.append([[1], [0]])
    else:
        target_data.append([[0], [1]])

network = NeuralNetwork()
network.add_layer(3, 1, "tanh")
network.add_layer(1, 2, "tanh")
network.add_layer(2, 2, "swish")

for i in range(0, 800):
    for j in range(0,len(train_data)):
        print("----------------")
        print("Input_data:\n" + str(train_data[j]))
        print("Forward pass:\n" + str(network.full_forward_propagation(train_data[j])))
        print("Target Data:\n" + str(target_data[j]))
        network.back_propagation(train_data[j], target_data[j])
print("Prediction : ", network.predict([0,0,2]))



----------------
Input_data:
[[3]
 [2]
 [1]]
Forward pass:
[[-0.04473316]
 [ 1.01055241]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[5]
 [1]
 [1]]
Forward pass:
[[-0.04440637]
 [ 1.01007662]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[5]
 [0]
 [0]]
Forward pass:
[[-0.03942208]
 [ 0.96891032]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [0]
 [1]]
Forward pass:
[[-0.04726835]
 [ 0.9382152 ]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 4]
 [-1]
 [ 2]]
Forward pass:
[[-0.0834258 ]
 [ 0.91806346]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 3]
 [-1]
 [ 5]]
Forward pass:
[[-0.21960623]
 [ 0.70065022]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 3]
 [-1]
 [11]]
Forward pass:
[[-0.23175695]
 [ 0.62009149]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [1]
 [3]]
Forward pass:
[[-0.05055808]
 [ 0.860129  ]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[-1]
 [ 1]
 [ 1]]
Forward pass:
[[-0.06572654]
 [

----------------
Input_data:
[[ 2]
 [-1]
 [ 6]]
Forward pass:
[[0.6936234 ]
 [0.30230755]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 1]
 [ 1]
 [-4]]
Forward pass:
[[0.73191954]
 [0.29161813]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 0]
 [ 0]
 [-1]]
Forward pass:
[[0.64291299]
 [0.31625158]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 2]
 [ 1]
 [-4]]
Forward pass:
[[0.70105359]
 [0.31538899]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 2]
 [ 1]
 [-1]]
Forward pass:
[[0.68196969]
 [0.32815865]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 1]
 [ 0]
 [-1]]
Forward pass:
[[0.64283931]
 [0.34450139]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[10]
 [ 4]
 [-2]]
Forward pass:
[[0.67618724]
 [0.3338241 ]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [2]
 [1]]
Forward pass:
[[0.68414451]
 [0.32752643]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[5]
 [1]
 [1]]
Forward pass:
[[0.66666326]
 [0.340195

 [-2]]
Forward pass:
[[0.68191777]
 [0.32620197]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [2]
 [1]]
Forward pass:
[[0.68989081]
 [0.32006548]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[5]
 [1]
 [1]]
Forward pass:
[[0.67200235]
 [0.33285721]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[5]
 [0]
 [0]]
Forward pass:
[[0.68035168]
 [0.32649448]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [0]
 [1]]
Forward pass:
[[0.68660939]
 [0.32069894]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 4]
 [-1]
 [ 2]]
Forward pass:
[[0.69550825]
 [0.314534  ]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 3]
 [-1]
 [ 5]]
Forward pass:
[[0.70053806]
 [0.30927336]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 3]
 [-1]
 [11]]
Forward pass:
[[0.70995167]
 [0.30328195]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [1]
 [3]]
Forward pass:
[[0.72038575]
 [0.29720376]]
Target Data:
[[1], [0]]
----------------
Inpu

----------------
Input_data:
[[ 1]
 [ 1]
 [-4]]
Forward pass:
[[0.72550099]
 [0.28566831]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 0]
 [ 0]
 [-1]]
Forward pass:
[[0.60945001]
 [0.32051122]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 2]
 [ 1]
 [-4]]
Forward pass:
[[0.70784941]
 [0.30658131]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 2]
 [ 1]
 [-1]]
Forward pass:
[[0.69141497]
 [0.31895375]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 1]
 [ 0]
 [-1]]
Forward pass:
[[0.64287188]
 [0.33901266]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[10]
 [ 4]
 [-2]]
Forward pass:
[[0.68630752]
 [0.32453452]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [2]
 [1]]
Forward pass:
[[0.69431076]
 [0.31832947]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[5]
 [1]
 [1]]
Forward pass:
[[0.67599071]
 [0.33137261]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[5]
 [0]
 [0]]
Forward pass:
[[0.68440829]
 [0.3249183 ]

----------------
Input_data:
[[ 2]
 [-1]
 [ 6]]
Forward pass:
[[0.74868822]
 [0.28228511]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 1]
 [ 1]
 [-4]]
Forward pass:
[[0.69794009]
 [0.29279726]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 0]
 [ 0]
 [-1]]
Forward pass:
[[0.56747259]
 [0.33604125]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 2]
 [ 1]
 [-4]]
Forward pass:
[[0.70797546]
 [0.30617451]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 2]
 [ 1]
 [-1]]
Forward pass:
[[0.6995609]
 [0.3164686]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 1]
 [ 0]
 [-1]]
Forward pass:
[[0.63828308]
 [0.34176415]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[10]
 [ 4]
 [-2]]
Forward pass:
[[0.69610713]
 [0.3214483 ]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [2]
 [1]]
Forward pass:
[[0.70408627]
 [0.31513437]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[5]
 [1]
 [1]]
Forward pass:
[[0.68489093]
 [0.32862698

...

----------------
Input_data:
[[ 2]
 [ 1]
 [-4]]
Forward pass:
[[-0.07187633]
 [ 1.00702972]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 2]
 [ 1]
 [-1]]
Forward pass:
[[0.06584967]
 [1.01371005]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 1]
 [ 0]
 [-1]]
Forward pass:
[[0.8818598 ]
 [0.03529599]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[10]
 [ 4]
 [-2]]
Forward pass:
[[0.95354113]
 [0.00611986]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [2]
 [1]]
Forward pass:
[[0.07156227]
 [1.00569909]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[5]
 [1]
 [1]]
Forward pass:
[[ 1.02453177]
 [-0.00778861]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[5]
 [0]
 [0]]
Forward pass:
[[ 1.02371804]
 [-0.00774755]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [0]
 [1]]
Forward pass:
[[ 1.02258673]
 [-0.00769307]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 4]
 [-1]
 [ 2]]
Forward pass:
[[ 1.02184197]
 [-0.00

Forward pass:
[[0.07078798]
 [1.00571567]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[5]
 [1]
 [1]]
Forward pass:
[[ 1.02432886]
 [-0.00761976]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[5]
 [0]
 [0]]
Forward pass:
[[ 1.02351906]
 [-0.00757911]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [0]
 [1]]
Forward pass:
[[ 1.02240099]
 [-0.00752693]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 4]
 [-1]
 [ 2]]
Forward pass:
[[ 1.02165958]
 [-0.00748415]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 3]
 [-1]
 [ 5]]
Forward pass:
[[ 1.02078002]
 [-0.00743544]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 3]
 [-1]
 [11]]
Forward pass:
[[ 1.01993685]
 [-0.007386  ]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [1]
 [3]]
Forward pass:
[[ 1.01777313]
 [-0.00728854]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[-1]
 [ 1]
 [ 1]]
Forward pass:
[[-0.07371627]
 [ 1.00041105]]
Target Data:
[[0], [1]]
---------

 [-0.00726079]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 3]
 [-1]
 [11]]
Forward pass:
[[ 1.01975714]
 [-0.00721283]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [1]
 [3]]
Forward pass:
[[ 1.01763499]
 [-0.00712454]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[-1]
 [ 1]
 [ 1]]
Forward pass:
[[-0.0728047 ]
 [ 1.00035054]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[2]
 [0]
 [1]]
Forward pass:
[[ 1.01691537]
 [-0.00707231]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[0]
 [0]
 [1]]
Forward pass:
[[0.90295877]
 [0.02441511]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [0]
 [5]]
Forward pass:
[[ 1.02114824]
 [-0.00719867]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 2]
 [-1]
 [ 6]]
Forward pass:
[[ 1.02029192]
 [-0.00715216]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 1]
 [ 1]
 [-4]]
Forward pass:
[[-0.07363028]
 [ 0.99714561]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 0]
 [ 

----------------
Input_data:
[[ 4]
 [-1]
 [ 2]]
Forward pass:
[[ 1.02125344]
 [-0.00712284]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 3]
 [-1]
 [ 5]]
Forward pass:
[[ 1.02039163]
 [-0.00707717]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 3]
 [-1]
 [11]]
Forward pass:
[[ 1.01956542]
 [-0.00703077]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [1]
 [3]]
Forward pass:
[[ 1.01748687]
 [-0.00695186]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[-1]
 [ 1]
 [ 1]]
Forward pass:
[[-0.07183298]
 [ 1.00029185]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[2]
 [0]
 [1]]
Forward pass:
[[ 1.0167742 ]
 [-0.00690126]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[0]
 [0]
 [1]]
Forward pass:
[[0.90417041]
 [0.02375719]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [0]
 [5]]
Forward pass:
[[ 1.02093444]
 [-0.00701568]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 2]
 [-1]
 [ 6]]
Forward pass:
[[ 1.02008738]
 

 [ 5]]
Forward pass:
[[ 1.02017955]
 [-0.006886  ]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 3]
 [-1]
 [11]]
Forward pass:
[[ 1.01936261]
 [-0.0068412 ]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [1]
 [3]]
Forward pass:
[[ 1.01732934]
 [-0.00677178]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[-1]
 [ 1]
 [ 1]]
Forward pass:
[[-0.07080637]
 [ 1.00023608]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[2]
 [0]
 [1]]
Forward pass:
[[ 1.01662412]
 [-0.00672288]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[0]
 [0]
 [1]]
Forward pass:
[[0.90544621]
 [0.02307586]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [0]
 [5]]
Forward pass:
[[ 1.02070845]
 [-0.00682524]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 2]
 [-1]
 [ 6]]
Forward pass:
[[ 1.01987117]
 [-0.00678182]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 1]
 [ 1]
 [-4]]
Forward pass:
[[-0.07157012]
 [ 0.99716789]]
Target Data:
[[0], [1]]
--